# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved.
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server.

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

In [ ]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [ ]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        self.incorrect_guesses = []
        self.pattern_cache = {}
        self.weight = [0.05, 0.1, 0.2, 0.25, 0.4]


        full_dictionary_location = "words_250000_train.txt"
        # test_dictionary_location = "words_test_hard.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        # self.test_dictionary = self.build_dictionary(test_dictionary_location)

        self.letter_set = sorted(set("".join(self.full_dictionary)))

        self.probabilities = [0] * len(self.letter_set)

        self.unigram, self.bigram, self.trigram, self.fourgram, self.fivegram = self.n_gram_freq(self.full_dictionary)

        self.tries_remaining = 6

        self.current_dictionary = []

        self.wins = []
        self.lose_words = []

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def normalize_pattern(self, word):
      return ''.join(word.split())

    # def get_adaptive_weights(pattern):
    #   known = sum(1 for c in pattern if c != '_' or c == ' ')
    #   total = len(pattern)

    #   if known <= 1:
    #       return (0.6, 0.25, 0.1, 0.03, 0.02)
    #   elif known == 2:
    #       return (0.4, 0.3, 0.2, 0.05, 0.05)
    #   elif known == 3:
    #       return (0.3, 0.3, 0.25, 0.1, 0.05)
    #   elif known == 4:
    #       return (0.25, 0.25, 0.25, 0.15, 0.1)
    #   else:
    #       return (0.2, 0.2, 0.2, 0.2, 0.2)

    def guess(self, word): # word input example: "_ p p _ e "

        n_word = self.normalize_pattern(word)

        self.incorrect_guesses = list(set(self.guessed_letters) - set(word))

        if len(self.guessed_letters) > 0 and self.guessed_letters[-1] in self.incorrect_guesses:
            self.update_ngram()

        if word in self.pattern_cache:
          self.probabilities = self.pattern_cache[word]
          return self.get_best_guess()

        self.probabilities = [0] * len(self.letter_set)
        clean_word = word[::2]

        # weight = get_adaptive_weights(word)
        # print(word)
        known = sum(1 for c in word if (c != '_' and c != ' '))
        # total = len(pattern)
        # print(known)

        if known <= 1:
            self.weight =  (0.6, 0.25, 0.1, 0.03, 0.02)
        elif known == 2:
            self.weight =  (0.4, 0.3, 0.2, 0.05, 0.05)
        elif known == 3:
            self.weight = (0.3, 0.3, 0.25, 0.1, 0.05)
        elif known == 4:
            self.weight = (0.25, 0.25, 0.25, 0.15, 0.1)
        else:
            self.weight = (0.2, 0.2, 0.2, 0.2, 0.2)
        # print(self.weight)
        return self.fivegram_probs(clean_word)

    def get_best_guess(self):
      maxi_prob = -1
      best_letter = None
      for i, p in enumerate(self.probabilities):
        if self.letter_set[i] not in self.guessed_letters and p > maxi_prob:
            maxi_prob = p
            best_letter = self.letter_set[i]
      return best_letter

    def n_gram_freq(self,words):
      unigram = collections.defaultdict(lambda: collections.defaultdict(int))
      bi_gram = collections.defaultdict(lambda: collections.defaultdict(lambda: collections.defaultdict(int)))
      tri_gram = collections.defaultdict(lambda: collections.defaultdict(lambda: collections.defaultdict(int)))
      four_gram = collections.defaultdict(lambda:collections.defaultdict(lambda: collections.defaultdict(lambda: collections.defaultdict(int))))
      five_gram = collections.defaultdict(lambda: collections.defaultdict(lambda:collections.defaultdict(lambda: collections.defaultdict(lambda: collections.defaultdict(int)))))

      for w in words:

        for i in range(len(w)-4):
          bi_gram[len(w)][w[i]][w[i+1]] += 1
          tri_gram[w[i]][w[i+1]][w[i+2]] += 1
          four_gram[w[i]][w[i+1]][w[i+2]][w[i+3]] +=1
          five_gram[w[i]][w[i+1]][w[i+2]][w[i+3]][w[i+4]] += 1
        curr = len(w)-4

        if(len(w)==2):
          bi_gram[len(w)][w[0]][w[1]] += 1
        elif (len(w)==3):
          bi_gram[len(w)][w[0]][w[1]] += 1
          bi_gram[len(w)][w[1]][w[2]] += 1
          tri_gram[w[0]][w[1]][w[2]] += 1
        elif len(w)>=4:
          bi_gram[len(w)][w[curr]][w[curr+1]] += 1
          bi_gram[len(w)][w[curr+1]][w[curr+2]] += 1
          bi_gram[len(w)][w[curr+2]][w[curr+3]] += 1
          tri_gram[w[curr]][w[curr+1]][w[curr+2]] += 1
          tri_gram[w[curr+1]][w[curr+2]][w[curr+3]] += 1
          four_gram[w[curr]][w[curr+1]][w[curr+2]][w[curr+3]] += 1

      for letters in set(w):
        unigram[len(w)][letters] += 1
      return unigram, bi_gram, tri_gram, four_gram, five_gram

    def update_ngram(self):

        update_dict = [word for word in self.full_dictionary if not set(word).intersection(set(self.incorrect_guesses))]
        self.unigram, self.bigram, self.trigram, self.fourgram, self.fivegram = self.n_gram_freq(update_dict)


    def fivegram_probs(self, word):


        total_count = 0
        letter_count = [0] * len(self.letter_set)

        for i in range(len(word) - 4):

            # "letter letter letter letter blank"
            if word[i] != '_' and word[i+1] != '_' and word[i+2] != '_' and word[i+3] != '_' and word[i+4] == '_':
                letter_1 = word[i]
                letter_2 = word[i+1]
                letter_3 = word[i+2]
                letter_4 = word[i+3]

                for j, letter in enumerate(self.letter_set):
                    if self.fivegram[letter_1][letter_2][letter_3][letter_4][letter] > 0 and letter not in self.guessed_letters:
                        total_count += self.fivegram[letter_1][letter_2][letter_3][letter_4][letter]
                        letter_count[j] += self.fivegram[letter_1][letter_2][letter_3][letter_4][letter]

            #"letter letter letter blank letter"
            elif word[i] != '_' and word[i+1] != '_' and word[i+2] != '_' and word[i+3] == '_' and word[i+4] != '_':
                letter_1 = word[i]
                letter_2 = word[i+1]
                letter_3 = word[i+2]
                letter_4 = word[i+4]


                for j, letter in enumerate(self.letter_set):
                    if self.fivegram[letter_1][letter_2][letter_3][letter][letter_4] > 0 and letter not in self.guessed_letters:
                        total_count += self.fivegram[letter_1][letter_2][letter_3][letter][letter_4]
                        letter_count[j] += self.fivegram[letter_1][letter_2][letter_3][letter][letter_4]

            #letter letter blank letter letter
            elif word[i] != '_' and word[i+1] != '_' and word[i+2] == '_' and word[i+3] != '_' and word[i+4] != '_':
                letter_1 = word[i]
                letter_2 = word[i+1]
                letter_3 = word[i+3]
                letter_4 = word[i+4]

                for j, letter in enumerate(self.letter_set):
                    if self.fivegram[letter_1][letter_2][letter][letter_3][letter_4] > 0 and letter not in self.guessed_letters:
                        total_count += self.fivegram[letter_1][letter_2][letter][letter_3][letter_4]
                        letter_count[j] += self.fivegram[letter_1][letter_2][letter][letter_3][letter_4]

            #letter blank letter letter letter
            elif word[i] != '_' and word[i+1] == '_' and word[i+2] != '_' and word[i+3] != '_' and word[i+4] != '_':
                letter_1 = word[i]
                letter_2 = word[i+2]
                letter_3 = word[i+3]
                letter_4 = word[i+4]

                for j, letter in enumerate(self.letter_set):
                    if self.fivegram[letter_1][letter][letter_2][letter_3][letter_4] > 0 and letter not in self.guessed_letters:
                        total_count += self.fivegram[letter_1][letter][letter_2][letter_3][letter_4]
                        letter_count[j] += self.fivegram[letter_1][letter][letter_2][letter_3][letter_4]

            #blank letter letter letter letter
            elif word[i] == '_' and word[i+1] != '_' and word[i+2] != '_' and word[i+3] != '_' and word[i+4] != '_':
                letter_1 = word[i+1]
                letter_2 = word[i+2]
                letter_3 = word[i+3]
                letter_4 = word[i+4]

                for j, letter in enumerate(self.letter_set):
                    if self.fivegram[letter][letter_1][letter_2][letter_3][letter_4] > 0 and letter not in self.guessed_letters:
                        total_count += self.fivegram[letter][letter_1][letter_2][letter_3][letter_4]
                        letter_count[j] += self.fivegram[letter][letter_1][letter_2][letter_3][letter_4]


        problty = [0] * len(self.letter_set)
        if total_count > 0:
            for i in range(len(self.letter_set)):
                problty[i] = letter_count[i] / total_count

        # interpolate probabilities between trigram and bigram
        for i, p in enumerate(self.probabilities):
            self.probabilities[i] = p + problty[i] * self.weight[4]
            #  (0.40)

        # run the next level down
        return self.fourgram_probs(word)

    def fourgram_probs(self, word):

        # vector of probabilities for each letter


        total_count = 0
        letter_count = [0] * len(self.letter_set)

        # traverse the word and find patterns that have three consecutive letters where one of them is blank
        for i in range(len(word) - 3):

            # case 1: "letter letter letter blank"
            if word[i] != '_' and word[i+1] != '_' and word[i+2] != '_' and word[i+3] == '_':
                letter_1 = word[i]
                letter_2 = word[i+1]
                letter_3 = word[i+2]

                # calculate occurences of "anchor_letter_1 anchor_letter_2 blank" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.fourgram[letter_1][letter_2][letter_3][letter] > 0 and letter not in self.guessed_letters:
                        total_count += self.fourgram[letter_1][letter_2][letter_3][letter]
                        letter_count[j] += self.fourgram[letter_1][letter_2][letter_3][letter]

            # case 2: "letter letter blank letter"
            elif word[i] != '_' and word[i+1] != '_' and word[i+2] == '_' and word[i+3] != '_':
                letter_1 = word[i]
                letter_2 = word[i+1]
                letter_3 = word[i+3]

                # calculate occurences of "anchor_letter_1 blank anchor_letter_2" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.fourgram[letter_1][letter_2][letter][letter_3] > 0 and letter not in self.guessed_letters:
                        total_count += self.fourgram[letter_1][letter_2][letter][letter_3]
                        letter_count[j] += self.fourgram[letter_1][letter_2][letter][letter_3]

            # case 3: letter blank letter letter
            elif word[i] != '_' and word[i+1] == '_' and word[i+2] != '_' and word[i+3] != '_':
                letter_1 = word[i]
                letter_2 = word[i+2]
                letter_3 = word[i+3]

                # calculate occurences of "blank anchor_letter_1 anchor_letter_2" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.fourgram[letter_1][letter][letter_2][letter_3] > 0 and letter not in self.guessed_letters:
                        total_count += self.fourgram[letter_1][letter][letter_2][letter_3]
                        letter_count[j] += self.fourgram[letter_1][letter][letter_2][letter_3]

            # case 4: blank letter letter letter
            elif word[i] == '_' and word[i+1] != '_' and word[i+2] != '_' and word[i+3] != '_':
                letter_1 = word[i+1]
                letter_2 = word[i+2]
                letter_3 = word[i+3]

                # calculate occurences of "blank anchor_letter_1 anchor_letter_2" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.fourgram[letter][letter_1][letter_2][letter_3] > 0 and letter not in self.guessed_letters:
                        total_count += self.fourgram[letter][letter_1][letter_2][letter_3]
                        letter_count[j] += self.fourgram[letter][letter_1][letter_2][letter_3]

        # calculate the probabilities of each letter appearing
        problty = [0] * len(self.letter_set)
        if total_count > 0:
            for i in range(len(self.letter_set)):
                problty[i] = letter_count[i] / total_count

        # interpolate probabilities between trigram and bigram
        for i, p in enumerate(self.probabilities):
            self.probabilities[i] = p + problty[i] * self.weight[3]
            #  (0.25)

        # run the next level down
        return self.trigram_probs(word)

    def trigram_probs(self, word):

        # vector of probabilities for each letter


        total_count = 0
        letter_count = [0] * len(self.letter_set)

        # traverse the word and find patterns that have three consecutive letters where one of them is blank
        for i in range(len(word) - 2):

            # case 1: "letter letter blank"
            if word[i] != '_' and word[i+1] != '_' and word[i+2] == '_':
                letter_1 = word[i]
                letter_2 = word[i+1]

                # calculate occurences of "anchor_letter_1 anchor_letter_2 blank" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.trigram[letter_1][letter_2][letter] > 0 and letter not in self.guessed_letters:
                        total_count += self.trigram[letter_1][letter_2][letter]
                        letter_count[j] += self.trigram[letter_1][letter_2][letter]

            # case 2: "letter blank letter"
            elif word[i] != '_' and word[i+1] == '_' and word[i+2] != '_':
                letter_1 = word[i]
                letter_2 = word[i+2]

                # calculate occurences of "anchor_letter_1 blank anchor_letter_2" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.trigram[letter_1][letter][letter_2] > 0 and letter not in self.guessed_letters:
                        total_count += self.trigram[letter_1][letter][letter_2]
                        letter_count[j] += self.trigram[letter_1][letter][letter_2]

            # case 3: blank letter letter
            elif word[i] == '_' and word[i+1] != '_' and word[i+2] != '_':
                letter_1 = word[i+1]
                letter_2 = word[i+2]

                # calculate occurences of "blank anchor_letter_1 anchor_letter_2" and for each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.trigram[letter][letter_1][letter_2] > 0 and letter not in self.guessed_letters:
                        total_count += self.trigram[letter][letter_1][letter_2]
                        letter_count[j] += self.trigram[letter][letter_1][letter_2]

        # calculate the probabilities of each letter appearing
        problty = [0] * len(self.letter_set)
        if total_count > 0:
            for i in range(len(self.letter_set)):
                problty[i] = letter_count[i] / total_count

        # interpolate probabilities between trigram and bigram
        for i, p in enumerate(self.probabilities):
            self.probabilities[i] = p + problty[i] * self.weight[2]
            #  (0.20)

        # run the next level down
        return self.bigram_probs(word)


    def bigram_probs(self, word):


        total_count = 0
        letter_count = [0] * len(self.letter_set)

        # traverse the word and find either patterns of "letter blank" or "blank letter"
        for i in range(len(word) - 1):
            # case 1: "letter blank"
            if word[i] != '_' and word[i+1] == '_':
                letter_1 = word[i]

                # calculate occurences of "anchor_letter blank" and each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.bigram[len(word)][letter_1][letter] > 0 and letter not in self.guessed_letters:
                        total_count += self.bigram[len(word)][letter_1][letter]
                        letter_count[j] += self.bigram[len(word)][letter_1][letter]

            # case 2: "blank letter"
            elif word[i] == '_' and word[i+1]!= '_':
                letter_1 = word[i+1]

                # calculate occurences of "blank anchor_letter" and each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.bigram[len(word)][letter][letter_1] > 0 and letter not in self.guessed_letters:
                        total_count += self.bigram[len(word)][letter][letter_1]
                        letter_count[j] += self.bigram[len(word)][letter][letter_1]

        # calculate the probabilities of each letter appearing
        problty = [0] * len(self.letter_set)
        if total_count > 0:
            for i in range(len(self.letter_set)):
                problty[i] = letter_count[i] / total_count

        # interpolate probabilities between trigram and bigram
        for i, p in enumerate(self.probabilities):
            self.probabilities[i] = p + problty[i] * self.weight[1]
            #  (0.10)

        # return letter associated with highest probability
        return self.unigram_probs(word)


    def unigram_probs(self, word):

        total_count = 0
        letter_count = [0] * len(self.letter_set)

        # traverse the word and find blank spaces
        for i in range(len(word)):
            # case 1: "blank"
            if word[i] == '_':
                # calculate occurences of pattern and each letter not guessed yet
                for j, letter in enumerate(self.letter_set):
                    if self.unigram[len(word)][letter] > 0 and letter not in self.guessed_letters:
                        total_count += self.unigram[len(word)][letter]
                        letter_count[j] += self.unigram[len(word)][letter]

        # calculate the probabilities of each letter appearing
        problty = [0] * len(self.letter_set)
        if total_count > 0:
            for i in range(len(self.letter_set)):
                problty[i] = letter_count[i] / total_count

        # interpolate probabilities
        for i, p in enumerate(self.probabilities):
            self.probabilities[i] = p + problty[i] * self.weight[0]
            #  (0.05)

        # adjust probabilities so they sum to one (not necessary but looks better)
        final_probs = [0] * len(self.letter_set)
        if sum(self.probabilities) > 0:
            for i in range(len(self.probabilities)):
                final_probs[i] = self.probabilities[i] / sum(self.probabilities)

        self.probabilities = final_probs

        # find letter with largest probability
        max_prob = 0
        guess_letter = ''
        for i, letter in enumerate(self.letter_set):
            if self.probabilities[i] > max_prob:
                max_prob = self.probabilities[i]
                guess_letter = letter

        # if no letter chosen from above, pick a random one (extra weight on vowels)
        if guess_letter == '':
            letters = self.letter_set.copy()
            random.shuffle(letters)
            letters_shuffled = ['e','a','i','o','u'] + letters
            for letter in letters_shuffled:
                if letter not in self.guessed_letters:
                    return letter

        return guess_letter

    # def guess(self, word): # word input example: "_ p p _ e "
    #     ###############################################
    #     # Replace with your own "guess" function here #
    #     ###############################################

    #     # clean the word so that we strip away the space characters
    #     # replace "_" with "." as "." indicates any character in regular expressions

    #     clean_word = word[::2].replace("_",".")

    #     # find length of passed word
    #     len_word = len(clean_word)

    #     # grab current dictionary of possible words from self object, initialize new possible words dictionary to empty
    #     current_dictionary = self.current_dictionary
    #     new_dictionary = []

    #     # iterate through all of the words in the old plausible dictionary
    #     for dict_word in current_dictionary:
    #         # continue if the word is not of the appropriate length
    #         if len(dict_word) != len_word:
    #             continue

    #         # if dictionary word is a possible match then add it to the current dictionary
    #         if re.match(clean_word,dict_word):
    #             new_dictionary.append(dict_word)

    #     # overwrite old possible words dictionary with updated version
    #     self.current_dictionary = new_dictionary


    #     # count occurrence of all characters in possible word matches
    #     full_dict_string = "".join(new_dictionary)

    #     c = collections.Counter(full_dict_string)
    #     sorted_letter_count = c.most_common()

    #     guess_letter = '!'

    #     # return most frequently occurring letter in all possible words that hasn't been guessed yet
    #     for letter,instance_count in sorted_letter_count:
    #         if letter not in self.guessed_letters:
    #             guess_letter = letter
    #             break

    #     # if no word matches in training dictionary, default back to ordering of full dictionary
    #     if guess_letter == '!':
    #         sorted_letter_count = self.full_dictionary_common_letter_sorted
    #         for letter,instance_count in sorted_letter_count:
    #             if letter not in self.guessed_letters:
    #                 guess_letter = letter
    #                 break

    #     return guess_letter

    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################

    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary


        # if verbose:
        #   print(f"Word to guess: {game.get_state_string()} ({len(word)} letters)")


          # Get model prediction, with enhanced strategy using correct/incorrect letters


        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            self.tries_remaining = tries_remains

            self.guessed_letters = []
            self.incorrect_guesses = []
            self.probabilities = [0] * len(self.letter_set)
            self.update_ngram()

            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code

                guess_letter = self.guess(word)

                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))

                try:
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e

                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"

    def my_status(self):
        return self.request("/my_status", {})

    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["error_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object.
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [ ]:
api = HangmanAPI(access_token="3a792201a081a919648e482adcb3dd", timeout=2000)

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [ ]:
for i in range(1000):
  api.start_game(practice=1,verbose=False)
  if(i%51==0):
    time.sleep(0.5)
  # [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)

[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status()
practice_success_rate = total_practice_successes / total_practice_runs
print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


In [ ]:
api.my_status()

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    #api.start_game(practice=0,verbose=False)

    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [ ]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)